In [ ]:
!sudo /opt/conda/bin/pip install mlflow==2.13.0
!sudo /opt/conda/bin/pip install pydantic
!sudo /opt/conda/bin/pip install starlette
!sudo /opt/conda/bin/pip install openai
!sudo /opt/conda/bin/pip install tiktoken


In [ ]:
from mlflow.deployments import get_deploy_client
import os

client = get_deploy_client(os.environ['DOMINO_MLFLOW_DEPLOYMENTS'])

response = client.predict(
	endpoint="sw-completions", 
	inputs={"prompt": "It's one small step for",
            #"n":2, 
            #"temperature":0.5, 
            #"max_tokens":50, 
            #"stop":[]
           }
)
response

In [ ]:
from mlflow.deployments import get_deploy_client
import os

client = get_deploy_client(os.environ['DOMINO_MLFLOW_DEPLOYMENTS'])

response = client.predict(
	endpoint="sw-completions",
	#inputs={"messages": [{"role": "user", "content": "Tell me a joke about rabbits"}]}
    inputs={"prompt": "Tell me a joke about rabbits"}
)
response

In [ ]:
import mlflow
professionalism_example_score_2 = mlflow.metrics.genai.EvaluationExample(
    input="What is MLflow?",
    output=(
        "MLflow is like your friendly neighborhood toolkit for managing your machine learning projects. It helps "
        "you track experiments, package your code and models, and collaborate with your team, making the whole ML "
        "workflow smoother. It's like your Swiss Army knife for machine learning!"
    ),
    score=2,
    justification=(
        "The response is written in a casual tone. It uses contractions, filler words such as 'like', and "
        "exclamation points, which make it sound less professional. "
    ),
)
professionalism_example_score_4 = mlflow.metrics.genai.EvaluationExample(
    input="What is MLflow?",
    output=(
        "MLflow is an open-source platform for managing the end-to-end machine learning (ML) lifecycle. It was "
        "developed by Databricks, a company that specializes in big data and machine learning solutions. MLflow is "
        "designed to address the challenges that data scientists and machine learning engineers face when "
        "developing, training, and deploying machine learning models.",
    ),
    score=4,
    justification=("The response is written in a formal language and a neutral tone. "),
)
professionalism_example_score_2 = mlflow.metrics.genai.EvaluationExample(
    input="What is MLflow?",
    output=(
        "MLflow is like your friendly neighborhood toolkit for managing your machine learning projects. It helps "
        "you track experiments, package your code and models, and collaborate with your team, making the whole ML "
        "workflow smoother. It's like your Swiss Army knife for machine learning!"
    ),
    score=2,
    justification=(
        "The response is written in a casual tone. It uses contractions, filler words such as 'like', and "
        "exclamation points, which make it sound less professional. "
    ),
)
professionalism_example_score_4 = mlflow.metrics.genai.EvaluationExample(
    input="What is MLflow?",
    output=(
        "MLflow is an open-source platform for managing the end-to-end machine learning (ML) lifecycle. It was "
        "developed by Databricks, a company that specializes in big data and machine learning solutions. MLflow is "
        "designed to address the challenges that data scientists and machine learning engineers face when "
        "developing, training, and deploying machine learning models.",
    ),
    score=4,
    justification=("The response is written in a formal language and a neutral tone. "),
)
professionalism = mlflow.metrics.genai.make_genai_metric(
    name="professionalism",
    definition=(
        "Professionalism refers to the use of a formal, respectful, and appropriate style of communication that is "
        "tailored to the context and audience. It often involves avoiding overly casual language, slang, or "
        "colloquialisms, and instead using clear, concise, and respectful language."
    ),
    grading_prompt=(
        "Professionalism: If the answer is written using a professional tone, below are the details for different scores: "
        "- Score 0: Language is extremely casual, informal, and may include slang or colloquialisms. Not suitable for "
        "professional contexts."
        "- Score 1: Language is casual but generally respectful and avoids strong informality or slang. Acceptable in "
        "some informal professional settings."
        "- Score 2: Language is overall formal but still have casual words/phrases. Borderline for professional contexts."
        "- Score 3: Language is balanced and avoids extreme informality or formality. Suitable for most professional contexts. "
        "- Score 4: Language is noticeably formal, respectful, and avoids casual elements. Appropriate for formal "
        "business or academic settings. "
    ),
    examples=[professionalism_example_score_2, professionalism_example_score_4],
    #model="openai:/gpt-3.5-turbo-16k", #This one gives the correct response 
    #model = "endpoints:/http://127.0.0.1:8767/gateway/sw-completions/invocations", #This one fails and so does the next one
    model = "endpoints:/sw-completions",
    parameters={"temperature": 0.0},
    aggregations=["mean", "variance"],
    greater_is_better=True,
)

In [ ]:
#os.environ['OPENAI_API_KEY']=''
os.environ['MLFLOW_DEPLOYMENTS_TARGET']=os.environ['DOMINO_MLFLOW_DEPLOYMENTS']
import mlflow
import openai
import os
import pandas as pd
from getpass import getpass

#os.environ['MLFLOW_TRACKING_URI'] = 'http://127.0.0.1:8080/'
eval_data = pd.DataFrame(
    {
        "inputs": [
            "What is MLflow?",
            "What is Spark?",
        ],
        "ground_truth": [
            "MLflow is an open-source platform for managing the end-to-end machine learning (ML) "
            "lifecycle. It was developed by Databricks, a company that specializes in big data and "
            "machine learning solutions. MLflow is designed to address the challenges that data "
            "scientists and machine learning engineers face when developing, training, and deploying "
            "machine learning models.",
            "Apache Spark is an open-source, distributed computing system designed for big data "
            "processing and analytics. It was developed in response to limitations of the Hadoop "
            "MapReduce computing model, offering improvements in speed and ease of use. Spark "
            "provides libraries for various tasks such as data ingestion, processing, and analysis "
            "through its components like Spark SQL for structured data, Spark Streaming for "
            "real-time data processing, and MLlib for machine learning tasks",
        ],
    }
)
experiment = mlflow.set_experiment("Demo-Custom-LLM-Metrics")
with mlflow.start_run() as run:
    system_prompt = "Answer the following question in two sentences"
    # Wrap "gpt-4" as an MLflow model.
    logged_model_info = mlflow.openai.log_model(
        model="gpt-4",
        task=openai.chat.completions,
        artifact_path="model",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "{question}"},
        ],
    )

    # Use predefined question-answering metrics to evaluate our model.
    results = mlflow.evaluate(
        logged_model_info.model_uri,
        eval_data,
        targets="ground_truth",
        model_type="question-answering",
        extra_metrics=[mlflow.metrics.toxicity(), mlflow.metrics.latency(),professionalism],
    )
    print(f"See aggregated evaluation results below: \n{results.metrics}")

    # Evaluation result for each data record is available in `results.tables`.
    eval_table = results.tables["eval_results_table"]
    print(f"See evaluation table below: \n{eval_table}")

In [ ]:
import mlflow
import os
logged_model = 'runs:/06c8c99723614dc6969f6f44003d7d24/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd

out = loaded_model.predict(pd.DataFrame({"input": ["What is MLflow?","What is Hadoop?"]}))
print(out)

In [ ]:
import os
import pandas as pd
import mlflow
from mlflow.deployments import get_deploy_client
import os



def predict(data):
    

    client = get_deploy_client(os.environ['DOMINO_MLFLOW_DEPLOYMENTS'])
    payload = data.to_dict()
    response = client.predict(
            endpoint="sw-completions", 
            inputs={"prompt":payload["prompt"][0]}
        )
    return response
        


input_example = pd.DataFrame.from_dict(
    {"prompt": ["Where is the moon?"]}
)
signature = mlflow.models.infer_signature(
    input_example, ["Above our heads."]
)



In [ ]:
with mlflow.start_run():
    model_info = mlflow.pyfunc.log_model(
        python_model=predict,
        registered_model_name="sw-completions",
        artifact_path="sw-completions",
        input_example=input_example,
        signature=signature,
    )



In [ ]:
df = pd.DataFrame.from_dict(
    {
        "prompt": ["Tell me about Jupiter", "Tell me about Saturn"]
    }
)

loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

print(loaded_model.predict(df))

In [ ]:
from mlflow.deployments import get_deploy_client
import os

client = get_deploy_client(os.environ['DOMINO_MLFLOW_DEPLOYMENTS'])

response = client.predict(
	endpoint="obl-compta", 
	inputs={"prompt": "It's one small step for"}
)

In [ ]:
import requests
import os
body_json = {
    "endpointName":"sw-completions",
    "endpointType":"llm/v1/completions",
    "endpointPermissions":{"isEveryoneAllowed":False,"userIds":["6283a3966d4fd0362f8ba2a8"]},
    "modelProvider":"openai",
    "modelName":"gpt-4",
    "modelConfig":{"openai_api_key":""}
}
my_headers = {"Content-Type":"application/json","X-Domino-Api-Key":os.environ['DOMINO_USER_API_KEY']}

r = requests.post(url="https://prod-field.cs.domino.tech/api/aigateway/v1/endpoints",headers=my_headers,json=body_json)
##The above succeeds


In [ ]:

import requests
import os
body_json = {   
    "modelName":"gpt-4",
    "modelConfig":{"openai_api_key":""}
}
my_headers = {"Content-Type":"application/json","X-Domino-Api-Key":os.environ['DOMINO_USER_API_KEY']}

r = requests.patch(url="https://prod-field.cs.domino.tech/api/aigateway/v1/endpoints/sw-completions",headers=my_headers,json=body_json)
print(r.text)
r.status_code
## This fails with status code 400
##{"requestId":"93ad6841-9ce1-44ed-b509-88050494f857","errors":["domino.aigateway.implementation.domain.exceptions.GatewayEndpointAlreadyExistsException: AI Gateway endpoint with name sw-completions already exists."]}


In [ ]:
from mlflow.deployments import get_deploy_client
import os

client = get_deploy_client(os.environ['DOMINO_MLFLOW_DEPLOYMENTS'])

response = client.predict(
	endpoint="sw-completions", 
	inputs={"prompt": "What is mlflow?"}
)
response

In [ ]:
client.get_endpoint(endpoint="sw-completions")